In [11]:
# 结合复数和有理数
from math import sin, cos, pi
def add_complex_and_rational(c,r):
    return ComplexRI(c.real+r.numer/r.denom,c.imag)

def mul_complex_and_rational(c,r):
    r_magnitude,r_angle = r.numer/r.denom,0
    if r_magnitude < 0:
        r_magnitude,r_angle = -r.numer/r.denom,pi
    return ComplexMA(c.magnitude * r_magnitude,c.angle+r_angle)

def add_rational_and_complex(r,c):
    return add_complex_and_rational(c,r)
def mul_rational_and_complex(r,c):
    return mul_complex_and_rational(c,r)

class Number:
    def __add__(self,other):
        if self.type_tag == other.type_tag:
            return self.add(other)
        elif (self.type_tag,other.type_tag) in self.adders:
            return self.cross_apply(other,self.adders)
        
    def __mul__(self,other):
        if self.type_tag == other.type_tag:
            return self.mul(other)
        elif (self.type_tag,other.type_tag) in self.multipliers:
            return self.cross_apply(other,self.multipliers)

    def cross_apply(self,other,cross_fns):
        cross_fn = cross_fns[(self.type_tag,other.type_tag)]
        return cross_fn(self,other)
    
    adders = {('com','rat'):add_complex_and_rational,
              ('rat','com'):add_rational_and_complex}
    
    multipliers = {('com','rat'):mul_complex_and_rational,
                   ('rat','com'):mul_rational_and_complex}
    
class Complex(Number):
    def add(self,other):
        return ComplexRI(self.real+other.real,self.imag+other.imag)
    def mul(self,other):
        magnitude = self.magnitude * other.magnitude
        return ComplexMA(magnitude, self.angle+other.angle)


from math import atan2
class ComplexRI(Complex):
    def __init__(self,real,imag):
        self.real = real
        self.imag = imag
    
    @property    # 让下面定义的方法可以用和调用属性值一样语法调用(即不需要使用括号)
    def magnitude(self):
        return (self.real ** 2 + self.imag ** 2)**0.5
    @property
    def angle(self):
        return atan2(self.imag,self.real)
    def __repr__(self):
        return 'ComplexRI({0:g},{1:g})'.format(self.real,self.imag)


class ComplexMA(Complex):
    def __init__(self,magnitude,angle):
        self.magnitude = magnitude
        self.angle = angle
    
    @property
    def real(self):
        return self.magnitude * cos(self.angle)
    @property
    def imag(self):
        return self.magnitude * sin(self.angle)
    def __repr__(self):
        return 'ComplexMA({0:g},{1:g}*pi)'.format(self.magnitude,self.angle/pi)

from math import gcd
class Rational(Number):
    def __init__(self,numer,denom):
        g = gcd(numer,denom)
        self.numer = numer // g
        self.denom = denom // g
    def __repr__(self):
        return 'Rational({0},{1})'.format(self.numer,self.denom)
    # def __str__(self):     # print的时候,如果没有定义__str__方法,则调用__repr__方法
    #     # 友好的显示   
    #     return '{}/{}'.format(self.numer, self.denom)
    def add(self,other):
        nx,dx = self.numer,self.denom
        ny,dy = other.numer,other.denom
        return Rational(nx*dy+ny*dx,dx*dy)
    def mul(self,other):
        numer = self.numer*other.numer
        denom = self.denom*other.denom
        return Rational(numer,denom)    
    

Rational.type_tag = 'rat'
Complex.type_tag = 'com'



In [12]:
ComplexRI(1.5, 0) + Rational(3, 2)

ComplexRI(3,0)

In [13]:
Rational(-1, 2) * ComplexMA(4, pi/2)

ComplexMA(2,1.5*pi)

In [ ]:
# 我一开始自己写的Number类如下:
# class Number:
#     def __add__(self,other):
#         if self.type_tag == other.type_tag:
#             return self.add(other)
#         elif (self.type_tag,other.type_tag) in self.adders:
#             return self.adders[(self.type_tag,other.type_tag)](self,other)
        
#     def __mul__(self,other):
#         if self.type_tag == other.type_tag:
#             return self.mul(other)
#         elif (self.type_tag,other.type_tag) in self.multipliers:
#             return self.multipliers[(self.type_tag,other.type_tag)](self,other)

#     adders = {('com','rat'):add_complex_and_rational,
#               ('rat','com'):add_rational_and_complex}
    
#     multipliers = {('com','rat'):mul_complex_and_rational,
#                    ('rat','com'):mul_rational_and_complex}
# 虽然输出结果一致,但是可以看到,为了实现不同类型数据相加,我直接在__add__的返回中引入在adders中的查找对应的数据类型的机制,mul同理
# 和教材的参考代码相比,参考代码查找并调用交叉类型的函数封装在cross_apply中,更容易扩展(比如增加减法,除法)
# 并且更容易维护(如果要修改调用不同类型数据的函数,参考代码只需要在cross_apply上改,而我的代码需要对每个运算符都进行修改)
